In [ ]:
##Experimento 03
## En este experimento agregamos los identificadores de arasaac al modelo
## Formato identificados: pict_id

In [1]:
!python --version

import torch
print(torch.__version__)
print(torch.cuda.is_available())

!pip install transformers==4.50.0
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate==0.4.0
!pip install bleu
!pip install -U scikit-learn
!pip install nltk datasets
!pip install sacrebleu

Python 3.10.15
2.5.1+cu118
True
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import torch
import sacrebleu
import numpy as np
import nltk
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

os.environ["CUDA_VISIBLE_DEVICES"]="1,3"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

torch.cuda.empty_cache()


In [3]:
import json
lista_pictogramas_ids=[]
with open("ids_only_id.txt", 'r', encoding='utf-8') as archivo:
  lineas = archivo.readlines()
  for linea in lineas:
    lista_pictogramas_ids.append(linea.strip())

print(len(lista_pictogramas_ids))

19769


In [4]:
dataset_train = load_dataset('json', data_files='train_data.json')['train']
dataset_test = load_dataset('json', data_files='test_data.json')['train']
dataset_valid = load_dataset('json', data_files='validation_data.json')['train']

# Mostrar estadísticas básicas
print(dataset_train)
print(dataset_test)
print(dataset_valid)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 15362
})
Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 423
})
Dataset({
    features: ['id', 'oracion', 'traduccion'],
    num_rows: 422
})


In [7]:
model_checkpoint = "flax-community/spanish-t5-small" #"vgaraujov/t5-base-spanish"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

# Agregar nuevos tokens al tokenizer
existing_tokens = set(tokenizer.get_vocab().keys())
new_tokens = [token for token in lista_pictogramas_ids if token not in existing_tokens]
tokenizer.add_tokens(new_tokens)

model.resize_token_embeddings(len(tokenizer))

# Validar parámetros actualizados
print("After adding new tokens:")
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)
print(f"Sample IDs of new tokens: {new_token_ids[:10]}")

new_embeddings = model.get_input_embeddings().weight.data[new_token_ids]
print("Sample embeddings for new tokens:", new_embeddings[:5])

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Convertir los nuevos tokens a sus IDs
new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)

# Mostrar ejemplos de tokens agregados y sus IDs
for token, token_id in zip(new_tokens[:10], new_token_ids[:10]):
    print(f"Token: {token}, ID: {token_id}")

60,493,824 total parameters.
60,493,824 training parameters.
After adding new tokens:
70,608,896 total parameters.
70,608,896 training parameters.
Sample IDs of new tokens: [32103, 32104, 32105, 32106, 32107, 32108, 32109, 32110, 32111, 32112]
Sample embeddings for new tokens: tensor([[ 0.4681, -0.5677, -0.4633,  ...,  0.7039, -0.5305, -0.0855],
        [ 0.4681, -0.5677, -0.4633,  ...,  0.7039, -0.5305, -0.0855],
        [ 0.4681, -0.5677, -0.4633,  ...,  0.7039, -0.5305, -0.0855],
        [ 0.4681, -0.5677, -0.4633,  ...,  0.7039, -0.5305, -0.0855],
        [ 0.4681, -0.5677, -0.4633,  ...,  0.7039, -0.5305, -0.0855]])
Token: pict_2247_acera, ID: 32103
Token: pict_2247_vereda, ID: 32104
Token: pict_2248_agua, ID: 32105
Token: pict_2249_alfombra, ID: 32106
Token: pict_2250_almohada, ID: 32107
Token: pict_2251_ambulancia, ID: 32108
Token: pict_2252_anillo, ID: 32109
Token: pict_2253_antena, ID: 32110
Token: pict_2253_parabólica, ID: 32111
Token: pict_2254_araña, ID: 32112


In [8]:
model.to(torch.device('cuda'))
model.device

device(type='cuda', index=0)

In [9]:
max_input_length = 40
max_target_length = 40

batch_size=16
metric ="bleu"
model_name = "t5-xgen-finetuned"
evaluation_strategy = "epoch"
save_strategy="epoch"
overwrite_output_dir=True
learning_rate=10e-5
gradient_accumulation_steps=1
weight_decay=0.01
do_train=True
do_eval=True
save_total_limit=20
num_train_epochs=20
seed=42
predict_with_generate=True
fp16=True
metric_for_best_model="bleu"
load_best_model_at_end=True
generation_max_length = max_target_length
logging_strategy="epoch"
eval_accumulation_steps=2

In [10]:
def preprocess_function(examples):
    inputs = ["translate: " + oracion for oracion in examples['oracion']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")    
    labels = tokenizer(text_target=examples["traduccion"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
encoded_train = dataset_train.map(preprocess_function, batched=True)
encoded_test = dataset_test.map(preprocess_function, batched=True)
encoded_validation = dataset_valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/15362 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

Map:   0%|          | 0/422 [00:00<?, ? examples/s]

In [12]:
nltk.download("punkt", quiet=True)

bleu_metric = evaluate.load("bleu")
chrf_metric = evaluate.load('chrf')

def compute_metrics_v2(eval_preds):
    print(f"metrics v2")
    predictions, labels = eval_preds

    # Reemplazar -100 con pad_token_id tanto en predicciones como en etiquetas
    pad_token_id = tokenizer.pad_token_id
    predictions = np.where(predictions != -100, predictions, pad_token_id)
    labels = np.where(labels != -100, labels, pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
     
    # Asegurarse de que BLEU reciba cadenas completas, no listas de palabras
    preds = [" ".join(pred.split()) for pred in decoded_preds]
    refs = [[" ".join(label.split())] for label in decoded_labels]  # Lista de listas para referencias
    
    print(f"bleu_preds {preds}")
    print(f"bleu_refs {refs}")
    
    # Calcular BLEU
    try:
        bleu_result = bleu_metric.compute(predictions=preds, references=refs)
        bleu_score = {"bleu": bleu_result["bleu"], "precisions": bleu_result["precisions"]}
    except Exception as e:
        print(f"Error al calcular BLEU: {e}")
        bleu_score = {"bleu": 0.0}

    # Calcular CHRF++
    try:
        chrf_result = chrf_metric.compute(predictions=preds, references=refs)
        print(chrf_result)
        chrf_score = {"chrf": chrf_result["score"]}
    except Exception as e:
        print(f"Error al calcular CHRF++: {e}")
        chrf_score = {"chrf": 0.0}
        
    # Combinar resultados
    combined_results = {**bleu_score, **chrf_score}
    return combined_results

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy = evaluation_strategy,
    save_strategy = save_strategy,
    overwrite_output_dir = overwrite_output_dir,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size= batch_size,
    gradient_accumulation_steps= gradient_accumulation_steps,
    weight_decay= weight_decay,
    do_train= do_train,
    do_eval= do_eval,
    save_total_limit= save_total_limit,
    num_train_epochs= num_train_epochs,
    seed= seed,
    predict_with_generate= predict_with_generate,
    fp16= fp16,
    generation_max_length=generation_max_length,
    logging_strategy=logging_strategy,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_v2
)



C:\ProgramData\anaconda3\envs\thesis\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\apare\AppData\Local\Temp\ipykernel_1016\3537574832.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
import numpy as np

import nltk
nltk.download('punkt')

trainer.train()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\apare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Precisions,Chrf
1,2.313600,1.536618,0.000000,"[0.5303030303030303, 0.0, 0.0, 0.0]",3.162051
2,1.498500,1.525631,0.000000,"[0.47067390440884416, 0.15624123422159888, 0.003871351995235259, 0.0]",23.457477
3,1.268200,1.570599,0.020401,"[0.47700691121743755, 0.15856921560343615, 0.0035901271503365746, 0.0006378568011481422]",23.359888
4,1.198200,1.551514,0.069011,"[0.49052517596101786, 0.17587939698492464, 0.019688644688644688, 0.013352874124735386]",26.722280
5,1.130500,1.571699,0.082618,"[0.4977675551346232, 0.18642365097588978, 0.027303233679072607, 0.018388934092758342]",28.357867
6,1.068000,1.551947,0.087174,"[0.5002727024815926, 0.18709304008103025, 0.029094827586206896, 0.021206641459806018]",29.164744
7,1.011000,1.560906,0.104359,"[0.5058014031300594, 0.2004578623551295, 0.041387705416920266, 0.028265107212475632]",30.281895
8,0.952500,1.600681,0.123661,"[0.5183818504851715, 0.21424427038004062, 0.0552639703612226, 0.03809995051954478]",31.735456
9,0.892800,1.609242,0.159216,"[0.5299097848716169, 0.23695075199056326, 0.0832417150934506, 0.06148160591298505]",33.351477
10,0.835900,1.612631,0.226505,"[0.553367304778956, 0.2867797601637906, 0.14412451361867704, 0.11508398469981707]",37.733609


Trainer is attempting to log a value of "[0.5303030303030303, 0.0, 0.0, 0.0]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


metrics v2
bleu_preds ['', 'yo borro', '', '', '', 'proteger', '', '', '', '', '', '', '', '', '', '', 'observar', '', '', '', '', '', 'pobreza', '', 'a rural peligroso', '', 'cáscara', '', '', 'responder', 'evita distraerte', '', '', '', '', '', '', '', 'responder', 'propongo', 'ingredientes', '', 'presentación clase', '', '', '', 'objetos', '', '', '', '', 'prohibido prohibido', '', '', '', '', '', '', '', '', '', 'prohibido prohibido', 'relaciona', '', '', '', '', '', '', '', '', '', '', '', '', 'juntarnos', '', 'energía eólica', '', '', '', '', '', 'reirme reirme', '', '', '', 'extinción', '', '', '', 'derechos', 'dia', '', '', '', '', '', '', '', 'inflamación', '', '', 'comparamos', '', '', '', '', '', '', '', '', '', '', '', '', '', 'cuánto', 'recorta', '', '', '', '', '', '', '', 'colarines', '', '', '', '', '', '', '', '', 'mina', '', '', '', '', '', 'noticia informativa', 'destinatario', 'ausencia', '', '', 'ahorras', '', '', '', '', '', '', 'cosechar', '', '', '', '', '', '',

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

...

{'score': 40.474636684374524, 'char_order': 6, 'word_order': 0, 'beta': 2}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

metrics v2
bleu_preds ['pict_2248_agua pict_11691_cuidar', 'pict_11691_cuidar borro pict_7029_la pict_9870_exposición pict_7074_de pict_11572_plantas', 'pict_3415_¿ pict_12281_tu pict_4649_feria pict_3418_?', 'pict_3415_¿ pict_22619_cómo pict_5581_ser pict_3418_?', 'pict_11749_hacer pict_8474_un pict_9870_exposición pict_7074_de pict_12281_tu pict_9897_información', 'pict_11691_cuidar pict_8476_el pict_2248_agua', 'pict_12281_tu pict_9870_exposición pict_7074_de pict_15485_usar', 'pict_11572_plantas pict_7034_en pict_9870_exposición pict_7271_tener pict_8579_explicar', 'pict_8579_explicar pict_7029_la pict_2248_agua', 'pict_3415_¿ pict_22619_cómo pict_2380_escribir pict_3418_?', 'pict_11691_cuidar pict_3047_y pict_11691_cuidar', 'pict_8579_explicar pict_5581_ser pict_5358_final', 'pict_2380_escribir pict_7029_la pict_37340_texto pict_7074_de pict_7033_los pict_9819_lugar', 'pict_2380_escribir pict_8474_una idea', 'pict_3415_¿ pict_7212_por pict_22624_qué pict_3418_?', 'pict_8579_explic

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=19220, training_loss=0.9377998566404218, metrics={'train_runtime': 11754.6545, 'train_samples_per_second': 26.138, 'train_steps_per_second': 1.635, 'total_flos': 1624313089228800.0, 'train_loss': 0.9377998566404218, 'epoch': 20.0})

In [15]:
## Se necesita saber con exactitud como se llama el folder para cargar el mejor modelo

tokenizerv1 = AutoTokenizer.from_pretrained('./results/checkpoint-19220')
modelv1 = AutoModelForSeq2SeqLM.from_pretrained('./results/checkpoint-19220')

device = torch.device("cuda")
modelv1.to(device)
modelv1.eval()

T5ForConditionalGeneration(
  (shared): Embedding(51859, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(51859, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [32]:
!pip install matplotlib

# Función para preparar los datos para la predicción
def prepare_data_for_prediction(dataset, tokenizer):
    # Transformar los datos para que sean aptos para el modelo
    prepared_data = []
    for example in dataset:
        inputs = tokenizerv1("translate: " + example['oracion'], return_tensors="pt", padding="max_length", truncation=True, max_length=max_input_length)
        inputs = {k: v.to(device1) for k, v in inputs.items()}
        prepared_data.append(inputs)
    return prepared_data

# Preparar los datos de validación para la predicción
prepared_validation_data = prepare_data_for_prediction(dataset_valid, tokenizerv1)

# Lista para guardar las predicciones
predictions = []

# Generar predicciones para cada entrada en los datos de validación
for inputs in prepared_validation_data:
    with torch.no_grad():
        outputs = modelv1.generate(**inputs, max_length=max_target_length)
    decoded_output = tokenizerv1.decode(outputs[0], skip_special_tokens=True)
    predictions.append(decoded_output)

# Imprimir las predicciones
for i, prediction in enumerate(predictions):
    print(f"Oracion: {dataset_valid[i]}, Prediction {i+1}: {prediction}")

Prediction 1: 
Prediction 2: 
Prediction 3: organizar 
Prediction 4: 
Prediction 5: 
Prediction 6: ordena 
Prediction 7: responder 
Prediction 8: 
Prediction 9: exponemos 
Prediction 10: yo 
Prediction 11: visión borrosa
Prediction 12: 
Prediction 13: 
Prediction 14: 
Prediction 15: berg
Prediction 16: yo 
Prediction 17: 
Prediction 18: 
Prediction 19: protegemos 
Prediction 20: 
Prediction 21: 
Prediction 22: 
Prediction 23: 
Prediction 24: 
Prediction 25: 
Prediction 26: 
Prediction 27: 
Prediction 28: 
Prediction 29: lica
Prediction 30: tráeme 
Prediction 31: 
Prediction 32: 
Prediction 33: corrige 
Prediction 34: 
Prediction 35: 
Prediction 36: 
Prediction 37: 
Prediction 38: 
Prediction 39: elaborar 
Prediction 40: 
Prediction 41: subrayo 
Prediction 42: 
Prediction 43: 
Prediction 44: 
Prediction 45: 
Prediction 46: 
Prediction 47: 
Prediction 48: 
Prediction 49: 
Prediction 50: 
Prediction 51: cartulinas 
Prediction 52: observa 
Prediction 53: 
Prediction 54: 
Prediction 55: 
Pr